In [8]:
import random

res = []

#generating correct data 
#always in range of 19 - 21
#we assume coverage = 20

for j in range(1,800001):
    res.append(random.randint(19,21))
    
print (len(res))

800000


In [9]:
duplication = []
deletion = []

#size of outliers
len_outliers = 2000
#generating 2000 duplactions to be insterted  at various positions 
#range of reads 100 - 500
for i in range(1,len_outliers+1):
    duplication.append(random.randint(100,500))

#generating 2000 deletion to be insterted  at various positions 
#range of reads 0 - 10
for i in range(1,len_outliers+1):
    deletion.append(random.randint(0,8))    
   
#fucntion to insert outliers to the original list
def insert_position(position, list):
    res[position:position+len_outliers] = list

#inserting duplication at differnet fixed location of the list
#starting indices of insertion for duplication
indices_dup = [0,15000,45000,110000,550000,795000]
for i in indices_dup:
    insert_position(i,duplication)
    
#inserting deletion at differnet fixed location of the list
#starting indices of insertion for deletion
indices_del = [6000,30000,70000,150000,685000]
for i in indices_del:
    insert_position(i,deletion)
    

print(len(res))



800000


In [10]:
#calculating mean median std iqr(inter quartile range) per sub window (size = 10bp)
import numpy as np 
subWindowMean = []
subWindowMedian = []
subWindowSTD = []
subWindowIQR = []
for i in range(0,len(res),10):
    subWindowMean.append(np.mean(res[i:i+10]))
    subWindowMedian.append(np.median(res[i:i+10]))
    subWindowSTD.append(np.std(res[i:i+10]))
    iqr = np.subtract(*np.percentile(res[i:i+10], [75, 25]))
    subWindowIQR.append(iqr)


In [11]:
#forming vectors per window (size - 100bp)

windowName = []
mean = []
median = []
std = []
iqr = []
start_pos = []
class_name = []
k = 0
for i in range(0,len(subWindowMean),10):
    mean.append((np.sum(subWindowMean[i:i+10]))/10)
    median.append((np.sum(subWindowMedian[i:i+10]))/10)
    std.append((np.sum(subWindowSTD[i:i+10]))/10)
    iqr.append((np.sum(subWindowIQR[i:i+10]))/10)
    start_pos.append(i*10)
    class_name.append('NOR')
    windowName.append('window'+str(k))
    k = k + 1



In [12]:
#assiging classes for duplication
for i in indices_dup:
    start_window_index=0
    end_window_index = 0
    for k in start_pos:
        if i in range(k,k+99):
             start_window_index = int(k/100)
        if (i+len_outliers) in range(k,k+100):
            end_window_index = int(k/100)
                
    for index in range(start_window_index,end_window_index):
        class_name[index] = "DUP"
            
#assiging classes for deletion
for i in indices_del:
    for k in start_pos:
        if i in range(k,k+99):
             start_window_index = int(k/100)
        if (i+len_outliers) in range(k,k+100):
            end_window_index = int(k/100)
                
    for index in range(start_window_index,end_window_index):
        class_name[index] = "DEL"


#randomly inserting impurities in dataset
l = ['DUP','DEL','NOR']
for i in range (0,20):
    x = random.randint(0,2)
    index = random.randint(0,8000)
    class_name[index] = l[x]

In [13]:

#creating the data set
import pandas as pd  

dict = {'window' : windowName, 
        'mean': mean, 'median': median, 
        'std': std, 'iqr' : iqr,
        'start_index' : start_pos,
        'class':class_name}  

df = pd.DataFrame(dict) 
# saving the dataframe 
df.to_csv('dataset.csv', index=False) 